In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/novedila/UAS_RFM.csv


In [2]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [3]:
datatrx = pd.read_csv('../input/novedila/UAS_RFM.csv', sep=';', parse_dates=['TrxDate'])
datatrx.head()

,CardID,TrxDate,Amount
0,1000152,2021-04-29,611634
1,1000158,2021-02-13,964322
2,1000156,2021-03-18,561552
3,1000115,2021-11-01,555993
4,1000164,2021-02-16,785855


In [4]:
datatrx.head(10)

,CardID,TrxDate,Amount
0,1000152,2021-04-29,611634
1,1000158,2021-02-13,964322
2,1000156,2021-03-18,561552
3,1000115,2021-11-01,555993
4,1000164,2021-02-16,785855
5,1000136,2021-06-24,504654
6,1000147,2021-01-05,505179
7,1000118,2021-06-19,401037
8,1000167,2021-04-25,344779
9,1000160,2021-08-02,442748


In [5]:
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [6]:
print(datatrx['TrxDate'].min(), datatrx['TrxDate'].max())

2021-01-02 00:00:00 2021-12-06 00:00:00


In [7]:
sd = dt.datetime(2021,12,18)
datatrx['hist']=sd - datatrx['TrxDate']
datatrx['hist'].astype('timedelta64[D]')
datatrx['hist']=datatrx['hist'] / np.timedelta64(1, 'D')
datatrx.head(10)

,CardID,TrxDate,Amount,hist
0,1000152,2021-04-29,611634,233.0
1,1000158,2021-02-13,964322,308.0
2,1000156,2021-03-18,561552,275.0
3,1000115,2021-11-01,555993,47.0
4,1000164,2021-02-16,785855,305.0
5,1000136,2021-06-24,504654,177.0
6,1000147,2021-01-05,505179,347.0
7,1000118,2021-06-19,401037,182.0
8,1000167,2021-04-25,344779,237.0
9,1000160,2021-08-02,442748,138.0


In [8]:
datatrx=datatrx[datatrx['hist'] < 730]
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
 3   hist     199 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.8 KB


In [9]:
rfmTable = datatrx.groupby('CardID').agg({'hist': lambda x:x.min(), # Recency
                                        'CardID': lambda x: len(x), # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'CardID': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)
rfmTable.head(10)

,recency,frequency,monetary_value
CardID,,,
1000101,303.0,1,492438
1000102,106.0,3,1177053
1000103,75.0,1,409882
1000104,234.0,2,689650
1000105,104.0,5,1868528
1000106,166.0,2,1203800
1000107,17.0,3,1508920
1000108,172.0,3,1568628
1000109,108.0,2,633942


In [10]:
quartiles = rfmTable.quantile(q=[0.20,0.40,0.60])
print(quartiles, type(quartiles))

     recency  frequency  monetary_value
0.2     75.0        1.0        663051.0
0.4    139.0        2.0        916750.0
0.6    188.0        3.0       1383101.0 <class 'pandas.core.frame.DataFrame'>


In [11]:
def RClass(x,p,d):
    if x <= d[p][0.20]:
        return 4
    elif x <= d[p][0.40]:
        return 3
    elif x <= d[p][0.60]: 
        return 2
    else:
        return 1
    
def FMClass(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    else:
        return 4

In [12]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary_value'].apply(FMClass, args=('monetary_value',quartiles,))

In [13]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,303.0,1,492438,1,1,1,111
1000102,106.0,3,1177053,3,3,3,333
1000103,75.0,1,409882,4,1,1,411
1000104,234.0,2,689650,1,2,2,122
1000105,104.0,5,1868528,3,4,4,344
1000106,166.0,2,1203800,2,2,3,223
1000107,17.0,3,1508920,4,3,4,434
1000108,172.0,3,1568628,2,3,4,234
1000109,108.0,2,633942,3,2,1,321


In [14]:
print("Pelanggan Tetap: ",len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Berlangganan: ',len(rfmSeg[rfmSeg['F_Quartile']==4]))
print("Pemborong: ",len(rfmSeg[rfmSeg['M_Quartile']==4]))
print("Beresiko: ",len(rfmSeg[rfmSeg['R_Quartile']==1]))
print('Nyaris Kehilangan Pelanggan: ', len(rfmSeg[rfmSeg['RFMClass']=='111']))
print('Kehilangan Pelanggan: ', len(rfmSeg[rfmSeg['RFMClass']=='333']) + len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Kehilangan Pelanggan Hemat: ', len(rfmSeg[rfmSeg['RFMClass']=='222']) + len(rfmSeg[rfmSeg['RFMClass']=='111']))

Pelanggan Tetap:  5
Berlangganan:  16
Pemborong:  28
Beresiko:  28
Nyaris Kehilangan Pelanggan:  8
Kehilangan Pelanggan:  7
Kehilangan Pelanggan Hemat:  9
